# Training an CRF-BILSTM for POS Tagging in Hausa
This code is used to train the Deep Learning Models for Hausa POS tagging. It requires the Python package `flair`. Per default it trains for 10 epochs and has an initial learning rate of 0.1.<br>


In [ ]:
% pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 401 kB 30.0 MB/s 
     |████████████████████████████████| 1.2 MB 51.8 MB/s 
     |████████████████████████████████| 101 kB 14.4 MB/s 
     |████████████████████████████████| 788 kB 65.2 MB/s 
     |████████████████████████████████| 19.7 MB 1.1 MB/s 
     |████████████████████████████████| 4.7 MB 55.2 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 981 kB 57.0 MB/s 
     |████████████████████████████████| 46 kB 1.9 MB/s 
     |████████████████████████████████| 1.6 MB 24.0 MB/s 
     |████████████████████████████████| 199 kB 77.5 MB/s 
     |████████████████████████████████| 62 kB 1.9 MB/s 
     |████████████████████████████████| 596 kB 33.9 MB/s 
     |████████████████████████████████| 6.6 MB 65.5 MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116702 sha256=5b9d1f50dbcbf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

from flair.data import Corpus, Sentence
from flair.datasets import ColumnCorpus
from flair.embeddings import TransformerWordEmbeddings, StackedEmbeddings, FastTextEmbeddings, BytePairEmbeddings
from flair.trainers import ModelTrainer
from flair.models import SequenceTagger
from flair import set_seed

We set a seed in order to makre results reproducible.

In [ ]:
seeds = [1103, 1704]
set_seed(seeds[1])

Next, we give the file path to the training data and the type of embeddings we want to use. Currently, this is `bert-bytepair` which uses multilingual BERT embeddings fine-tuned on Hausa and Byte-Pair encodings embeddings or `bert` which only used the multilingual BERT embeddings.<br>
Results will be stored in `OUT_PATH`. The test

In [ ]:
# This is the folder in which train, test and dev files reside.
DATA_PATH = 'PATH/TO/TRAIN-TEST-DEV'

# Either bert-bytepair, bert or afribert-bytepair
TYPE = "bert-bytepair"

# Training file
EXP_FILE = "Tanzil-en-fast_align_sym"

# Test file
test = "test.conll"

# This where the results will be stored (final model)
OUT_PATH = 'OUTPUT/PATH'

In [ ]:
# define columns, first one is assumed to be the tokens, the second colum contains the corresponding tags.
columns = {0: 'text', 1: 'pos'}

corpus: Corpus = ColumnCorpus(DATA_PATH, columns,
                              train_file=EXP_FILE+".conll",
                              test_file=test)

2022-08-11 14:54:47,934 Reading data from drive/MyDrive/Data/experiments
2022-08-11 14:54:47,938 Train: drive/MyDrive/Data/experiments/Tanzil-en-fast_align_sym.conll
2022-08-11 14:54:47,943 Dev: None
2022-08-11 14:54:47,945 Test: drive/MyDrive/Data/experiments/test.conll


In [ ]:
label_dict = corpus.make_label_dictionary(label_type="pos")
print(label_dict)

2022-08-11 14:54:54,595 Computing label dictionary. Progress:


7637it [00:00, 8259.41it/s]

2022-08-11 14:54:55,580 Dictionary created for label 'pos' with 16 values: VERB (seen 32089 times), PRON (seen 25321 times), PUNCT (seen 23826 times), NOUN (seen 19413 times), ADP (seen 17849 times), ADV (seen 9501 times), CCONJ (seen 9487 times), DET (seen 9303 times), PROPN (seen 6349 times), ADJ (seen 4575 times), AUX (seen 1881 times), PART (seen 397 times), NUM (seen 385 times), INTJ (seen 240 times), X (seen 6 times)
Dictionary with 16 tags: <unk>, VERB, PRON, PUNCT, NOUN, ADP, ADV, CCONJ, DET, PROPN, ADJ, AUX, PART, NUM, INTJ, X


Next, we choose the embeddings according to the specified type in the variable `TYPE`.

In [ ]:
if TYPE == "bert-bytepair":
  embedding_types = [
      TransformerWordEmbeddings("Davlan/bert-base-multilingual-cased-finetuned-hausa"),
      BytePairEmbeddings("ha")
  ]
elif TYPE == "afribert-bytepair":
    embedding_types = [
      TransformerWordEmbeddings("castorini/afriberta_large"),
      BytePairEmbeddings("ha")
  ]
elif TYPE == "bert":
    embedding_types = [
      TransformerWordEmbeddings("Davlan/bert-base-multilingual-cased-finetuned-hausa")
  ]
else:
  raise ValueError("Unknown embedding type: {}".format(TYPE))

BPEmb fallback: ha from vocab size 100000 to 5000
downloading https://nlp.h-its.org/bpemb/ha/ha.wiki.bpe.vs5000.model


100%|██████████| 315468/315468 [00:00<00:00, 1192339.27B/s]


downloading https://nlp.h-its.org/bpemb/ha/ha.wiki.bpe.vs5000.d50.w2v.bin.tar.gz


100%|██████████| 957581/957581 [00:00<00:00, 2138970.64B/s]


In [ ]:
embeddings = StackedEmbeddings(embeddings=embedding_types)

In [ ]:
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type="pos",
                        use_crf=True)

2022-08-11 14:55:20,615 SequenceTagger predicts: Dictionary with 16 tags: <unk>, VERB, PRON, PUNCT, NOUN, ADP, ADV, CCONJ, DET, PROPN, ADJ, AUX, PART, NUM, INTJ, X


Finally, training is started:

In [ ]:
trainer = ModelTrainer(tagger, corpus)

In [ ]:
trainer.train(OUT_PATH,
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=10,
              write_weights=True,
              patience=2)

2022-08-11 14:55:24,172 ----------------------------------------------------------------------------------------------------
2022-08-11 14:55:24,178 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): TransformerWordEmbeddings(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(119547, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linea

100%|██████████| 27/27 [00:24<00:00,  1.11it/s]

2022-08-11 15:02:01,436 Evaluating as a multi-label problem: False
2022-08-11 15:02:01,567 DEV : loss 1.1910310983657837 - f1-score (micro avg)  0.6486


2022-08-11 15:02:01,673 BAD EPOCHS (no improvement): 0
2022-08-11 15:02:01,680 saving best model
2022-08-11 15:02:04,894 ----------------------------------------------------------------------------------------------------
2022-08-11 15:02:23,723 epoch 2 - iter 23/239 - loss 1.23383781 - samples/sec: 39.11 - lr: 0.100000
2022-08-11 15:02:45,049 epoch 2 - iter 46/239 - loss 1.23372104 - samples/sec: 45.22 - lr: 0.100000
2022-08-11 15:03:06,716 epoch 2 - iter 69/239 - loss 1.23402671 - samples/sec: 44.21 - lr: 0.100000
2022-08-11 15:03:28,278 epoch 2 - iter 92/239 - loss 1.22508404 - samples/sec: 44.64 - lr: 0.100000
2022-08-11 15:03:50,247 epoch 2 - iter 115/239 - loss 1.22037242 - samples/sec: 43.71 - lr: 0.100000
2022-08-11 15:04:10,702 epoch 2 - iter 138/239 - loss 1.22190495 - samples/sec: 48.10 - lr: 0.100000
2022-08-11 15:04:32,174 epoch 2 - iter 161/239 - loss 1.21490918 - samples/sec: 44.79 - lr: 0.100000
2022-08-11 15:04:52,103 epoch 2 - iter 184/239 - loss 1.21382190 - samples/

100%|██████████| 27/27 [00:13<00:00,  2.01it/s]

...

100%|██████████| 27/27 [00:11<00:00,  2.35it/s]


2022-08-11 15:44:32,105 Evaluating as a multi-label problem: False
2022-08-11 15:44:32,235 DEV : loss 1.1549733877182007 - f1-score (micro avg)  0.6259
2022-08-11 15:44:32,330 BAD EPOCHS (no improvement): 2
2022-08-11 15:44:35,110 ----------------------------------------------------------------------------------------------------
2022-08-11 15:44:35,118 loading file drive/MyDrive/Data/resources_run2/Tanzil-en-fast_align_sym/bert-bytepair/best-model.pt
2022-08-11 15:44:39,701 SequenceTagger predicts: Dictionary with 18 tags: <unk>, VERB, PRON, PUNCT, NOUN, ADP, ADV, CCONJ, DET, PROPN, ADJ, AUX, PART, NUM, INTJ, X, <START>, <STOP>


100%|██████████| 8/8 [00:01<00:00,  4.58it/s]

2022-08-11 15:44:41,787 Evaluating as a multi-label problem: False
2022-08-11 15:44:41,813 0.4881	0.4881	0.4881	0.4881
2022-08-11 15:44:41,815 
Results:
- F-score (micro) 0.4881
- F-score (macro) 0.2857
- Accuracy 0.4881

By class:
              precision    recall  f1-score   support

        VERB     0.4467    0.9019    0.5975       418
        NOUN     0.8092    0.5491    0.6542       479
        PRON     0.3307    0.6522    0.4388       253
       PUNCT     0.7698    0.9180    0.8374       317
         AUX     0.7843    0.0864    0.1556       463
         ADP     0.4467    0.5906    0.5087       149
        PART     0.0000    0.0000    0.0000       286
         ADV     0.1636    0.3506    0.2231        77
         DET     0.4118    0.5957    0.4870        47
        CONJ     0.0000    0.0000    0.0000        89
        INTJ     0.6000    0.1463    0.2353        41
       CCONJ     0.0000    0.0000    0.0000         0
       PROPN     0.0000    0.0000    0.0000         0
         AD

{'dev_loss_history': [1.1910310983657837,
  1.0660001039505005,
  1.0288790464401245,
  0.9974625110626221,
  0.9877561330795288,
  1.406103253364563,
  1.2589526176452637,
  1.2056884765625,
  1.1722631454467773,
  1.1549733877182007],
 'dev_score_history': [0.6485683259987426,
  0.6793164542492999,
  0.6877178944961994,
  0.6934331599702807,
  0.6938903812082071,
  0.49877121792307255,
  0.591072755329485,
  0.6083328570612105,
  0.6211350517231525,
  0.62587872206664],
 'test_score': 0.4880726997349489,
 'train_loss_history': [1.6035722777026817,
  1.2002931522103817,
  1.1123701024498707,
  1.0538615171302324,
  1.010418737119166,
  1.6436094155406404,
  1.4370607127037296,
  1.360074660441129,
  1.308443386965496,
  1.284898236608602]}